In [1]:
import pandas as pd

# Load the provided CSV data
file_path = '../problem2/predicted_future_volumes_hours.csv'
predicted_volumes = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
predicted_volumes.head()

,分拣中心,日期时间,货量
0,分拣中心_SC1,2023/11/01 00:00,5679.73
1,分拣中心_SC1,2023/11/01 01:00,5130.17
2,分拣中心_SC1,2023/11/01 02:00,5216.39
3,分拣中心_SC1,2023/11/01 03:00,5019.86
4,分拣中心_SC1,2023/11/01 04:00,4220.94


In [2]:
# Convert the date-time column to datetime type for easier manipulation
predicted_volumes['日期时间'] = pd.to_datetime(predicted_volumes['日期时间'], format='%Y/%m/%d %H:%M')

# Extract date and hour for easier aggregation
predicted_volumes['日期'] = predicted_volumes['日期时间'].dt.date
predicted_volumes['小时'] = predicted_volumes['日期时间'].dt.hour

# Define the shifts based on the hour ranges
shifts = {
    (0, 8): '00:00-08:00',
    (5, 13): '05:00-13:00',
    (8, 16): '08:00-16:00',
    (12, 20): '12:00-20:00',
    (14, 22): '14:00-22:00',
    (16, 24): '16:00-24:00',
}

# Assign shifts to each hour
def assign_shift(hour):
    for start, end in shifts:
        if start <= hour < end:
            return shifts[(start, end)]
    return None

predicted_volumes['班次'] = predicted_volumes['小时'].apply(assign_shift)

# Group by sorting center, date, and shift, then sum up the volumes
grouped_data = predicted_volumes.groupby(['分拣中心', '日期', '班次']).agg({'货量': 'sum'}).reset_index()

grouped_data.head()

,分拣中心,日期,班次,货量
0,分拣中心_SC1,2023-11-01,00:00-08:00,34294.59
1,分拣中心_SC1,2023-11-01,05:00-13:00,21413.50
2,分拣中心_SC1,2023-11-01,08:00-16:00,11589.62
3,分拣中心_SC1,2023-11-01,12:00-20:00,15372.30
4,分拣中心_SC1,2023-11-01,14:00-22:00,8951.26


In [11]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

# Create the linear programming model
model = LpProblem("Staff_Scheduling", LpMinimize)

# Maximum efficiency in terms of packages per hour
efficiency_regular = 25
efficiency_temp = 20

# Add decision variables for regular and temporary workers
regular_workers = LpVariable.dicts("RegularWorkers",
                                   [(center, date, shift) for center, date, shift, _ in grouped_data.itertuples(index=False, name=None)],
                                   lowBound=0, cat='Integer')

temporary_workers = LpVariable.dicts("TemporaryWorkers",
                                     [(center, date, shift) for center, date, shift, _ in grouped_data.itertuples(index=False, name=None)],
                                     lowBound=0, cat='Integer')

# Objective: Minimize the total number of person-days
model += lpSum([regular_workers[(center, date, shift)] + temporary_workers[(center, date, shift)]
                for center, date, shift, _ in grouped_data.itertuples(index=False, name=None)])

# Constraint: Sufficient staff to handle the volume
for index, row in grouped_data.iterrows():
    model += (regular_workers[(row['分拣中心'], row['日期'], row['班次'])] * efficiency_regular +
              temporary_workers[(row['分拣中心'], row['日期'], row['班次'])] * efficiency_temp >= row['货量'])

# Constraint: Each staff member can only work one shift per day
for center, date in grouped_data[['分拣中心', '日期']].drop_duplicates().itertuples(index=False, name=None):
    for shift in shifts.values():
        model += (lpSum([regular_workers[(center, date, shift)], temporary_workers[(center, date, shift)]]) <= 1)

# Constraint: Priority for regular workers, total staff <= minimum required for each shift
for index, row in grouped_data.iterrows():
    model += (regular_workers[(row['分拣中心'], row['日期'], row['班次'])] <= 
              (row['货量'] / efficiency_regular + temporary_workers[(row['分拣中心'], row['日期'], row['班次'])]))
    
# Solve the model
model.solve()



# Check the status of the solution
if LpStatus[model.status] == "Optimal":
    print("Optimal solution found!")
else:
    print("No optimal solution found. Status:", LpStatus[model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5afbbef3797e476eacbdf83f27d7b103-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5afbbef3797e476eacbdf83f27d7b103-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31811 COLUMNS
At line 159036 RHS
At line 190843 BOUNDS
At line 212048 ENDATA
Problem MODEL has 31806 rows, 21204 columns and 63612 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.13 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.18   (Wallclock seconds):       0.23

No optimal solution found. Status: Infeasible


In [17]:
# Load the provided CSV data
file_path = '../problem2/predicted_future_volumes_hours.csv'
df = pd.read_csv(file_path)
# Add decision variables for regular and temporary workers for each hour and each sorting center
# Add decision variables for regular and temporary workers for each hour and each sorting center
regular_workers = LpVariable.dicts("RegularWorkers",
                                   [(center, date) for center, date in df[['分拣中心', '日期时间']].values],
                                   lowBound=0, cat='Integer')

temporary_workers = LpVariable.dicts("TemporaryWorkers",
                                     [(center, date) for center, date in df[['分拣中心', '日期时间']].values],
                                     lowBound=0, cat='Integer')

# Constraint: Ensure that the total number of processed packages in each hour equals the predicted volume
for center, date, volume in df.itertuples(index=False, name=None):
    for hour in range(24):  # Assuming each day has 24 hours
        model += regular_workers[(center, date)] * efficiency_regular + temporary_workers[(center, date)] * efficiency_temp == volume

# Objective: Minimize the total number of person-hours
model += lpSum([regular_workers[(center, date)] + temporary_workers[(center, date)]
                for center, date in df[['分拣中心', '日期时间']].values])

# Solve the model
model.solve()

# Check the status of the solution
if LpStatus[model.status] == "Optimal":
    print("Optimal solution found!")
    # Print the total number of person-hours
    print("Total person-hours:", value(model.objective))
    # Print the number of regular and temporary workers for each sorting center and each hour
    for var in model.variables():
        if var.varValue > 0:
            print(var.name, "=", var.varValue)
else:
    print("No optimal solution found. Status:", LpStatus[model.status])

/home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/pulp/pulp.py:1668: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ac4ec73a4a7a4faebd475c8f26c7bbbe-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/ac4ec73a4a7a4faebd475c8f26c7bbbe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2067395 COLUMNS
Duplicate row C0031806 at line 2067447 <     X0000001  C0031806   2.500000000000e+01 >
Duplicate row C0031807 at line 2067448 <     X0000001  C0031807   2.500000000000e+01 >
Duplicate row C0031808 at line 2067449 <     X0000001  C0031808   2.500000000000e+01 >
Duplicate row C0031809 at line 2067450 <     X0000001  C0031809   2.500000000000e+01 >
Duplicate row C0031810 at line 2067451 <     X0000001  C0031810   2.500000000000e+01 >
Duplicate row C0031811 at line 2067452 <     X0000001  C0031811   2.500000000000e+01 >
Duplicate row C0031812 at line 2067453 <     X00000

PulpSolverError: Pulp: Error while executing /home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc